* BOW - CoutnVectorizer

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

In [4]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [5]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [6]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
X = df['Message']
y = df['spam']

In [8]:
type(X),type(y)

(pandas.core.series.Series, pandas.core.series.Series)

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [10]:
X_train.shape,X_test.shape

((4457,), (1115,))

In [11]:
y_train.shape,y_test.shape

((4457,), (1115,))

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()
X_train_cv = v.fit_transform(X_train.values)

In [16]:
X_train_np = X_train_cv.toarray()
X_train_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4457, 7770))

In [20]:
X_train[:4]

5554    Well keep in mind I've only got enough gas for...
4312    It so happens that there r 2waxsto do wat you ...
4257    Aah! A cuddle would be lush! I'd need lots of ...
4411                 You also didnt get na hi hi hi hi hi
Name: Message, dtype: object

In [24]:
#5554 is the the index in original df
#iloc[0] given item at 0th postion after splitting

In [21]:
X_train.iloc[0]

"Well keep in mind I've only got enough gas for one more round trip barring a sudden influx of cash"

In [22]:
X_train[5554]

"Well keep in mind I've only got enough gas for one more round trip barring a sudden influx of cash"

In [27]:
X_train[:4][5554]

"Well keep in mind I've only got enough gas for one more round trip barring a sudden influx of cash"

In [29]:
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(7770,))

In [44]:
np.where(X_train_np[0]!=0)  #gives the non zero index

(array([1253, 1705, 2630, 2968, 3110, 3230, 3672, 3703, 3944, 4515, 4613,
        4927, 4972, 4978, 5859, 6586, 7061, 7281, 7474]),)

In [37]:
v.get_feature_names_out()[2630]

'enough'

In [62]:
non_zero = np.where(X_train_np[0]!=0)[0]
non_zero_list = non_zero.tolist()

In [65]:
[v.get_feature_names_out()[i] for i in non_zero_list]

['barring',
 'cash',
 'enough',
 'for',
 'gas',
 'got',
 'in',
 'influx',
 'keep',
 'mind',
 'more',
 'of',
 'one',
 'only',
 'round',
 'sudden',
 'trip',
 've',
 'well']

In [67]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_cv,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [68]:
from sklearn.metrics import classification_report
X_test_cv = v.transform(X_test)
y_pred = model.predict(X_test_cv)
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       984
           1       0.94      0.95      0.94       131

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [72]:
X_test[:10]

3358    Brainless Baby Doll..:-D;-), vehicle sariyag d...
3322                               Yo im right by yo work
2891    Shuhui has bought ron's present it's a swatch ...
3825    I miss you so much I'm so desparate I have rec...
760     Call me when you/carlos is/are here, my phone'...
3202                         Nice.nice.how is it working?
5327                        Wishing you a wonderful week.
1361    Yo dude guess who just got arrested the other day
297     Unless it's a situation where YOU GO GURL woul...
3144    Boy; I love u Grl: Hogolo Boy: gold chain kods...
Name: Message, dtype: object

In [74]:
y_pred[:12]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
check = ['Hello! this is a random good message melody!!',
         'Urgent!! Hurry up loosing $500'
        ]

check_cv = v.transform(check)
pred = model.predict(check_cv)
pred

array([0, 1])

### Using Pipeline

In [77]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('cv',CountVectorizer()),
    ('nb',MultinomialNB())
])

clf.fit(X_train,y_train)

,steps,"[('cv', ...), ('nb', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [78]:
y_pred = clf.predict(X_test)

report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       984
           1       0.94      0.95      0.94       131

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [79]:
check = ['Hello! this is a random good message melody!!',
         'Urgent!! Hurry up loosing $500'
        ]
y_pred = clf.predict(check)
y_pred

array([0, 1])